In [1]:
!pip install fasttext
import fasttext


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install datasets
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
train_dataset = imdb_dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = imdb_dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from string import punctuation
import re
def clean_text(text_data):
    text_data = text_data.lower()
    text_data = re.sub("[" + punctuation + "]( |$)", "", text_data)
    return text_data

with open('imdb_train.txt', 'w') as f:
    for example in train_df.shuffle(seed=42):
        label = '__label__' + ('positive' if example['label'] == 1 else 'negative')
        text = clean_text(example['text']).replace('\n', ' ')
        f.write(label + ' ' + text + '\n')

In [4]:
model = fasttext.train_supervised('imdb_train.txt')

In [5]:
with open('imdb_test.txt', 'w') as f:
    for example in test_df.shuffle(42):
        label = '__label__' + ('positive' if example['label'] == 1 else 'negative')
        text = clean_text(example['text']).replace('\n', ' ')
        f.write(label + ' ' + text + '\n')

In [6]:
result = model.test('imdb_test.txt')
result

(25000, 0.84172, 0.84172)

In [7]:
with open('imdb_val.txt', 'w') as f:
    for example in valid_df.shuffle(42):
        label = '__label__' + ('positive' if example['label'] == 1 else 'negative')
        text = clean_text(example['text']).replace('\n', ' ')
        f.write(label + ' ' + text + '\n')

In [ ]:
best_model = fasttext.train_supervised(
    input='imdb_train.txt',
    autotuneValidationFile='imdb_val.txt',
    autotuneMetric='accuracy')